# Shipping and Conflict

This notebook explores the relationship between trade and conflict in the Red Sea region. The focus is on the Beb El-Mandeb strait and the conflict in Yemen. 

In [2]:
import pandas as pd

In [6]:
df = pd.read_csv("../../data/portwatch/chokepoints_data_02_26_2024.csv")
bab_el_mandeb = df[
    (df["portname"] == "Bab el-Mandeb Strait") & (df["date"] > "2023-01-01")
]

In [12]:
bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])

C:\Users\sahit\AppData\Local\Temp\ipykernel_29100\3160702748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bab_el_mandeb['date'] = pd.to_datetime(bab_el_mandeb['date'])


In [22]:
bab_el_mandeb = (
    bab_el_mandeb.groupby(pd.Grouper(key="date", freq="W"))["n_total"]
    .sum()
    .reset_index()
)  # merge(conflict_by_country, on = 'date')

In [36]:
conflict_by_country = pd.read_csv(
    "../../data/conflict/conflict_by_country_2023-01-01_2024-02-27.csv"
)
conflict_by_port = pd.read_csv(
    "../../data/conflict/conflict_by_port_2023-01-01_2024-02-27.csv"
)

In [37]:
conflict_by_country["event_date"] = pd.to_datetime(conflict_by_country["event_date"])
conflict_by_country.rename(columns={"event_date": "date"}, inplace=True)

In [38]:
conflict_by_port["event_date"] = pd.to_datetime(conflict_by_port["event_date"])
conflict_by_port.rename(columns={"event_date": "date"}, inplace=True)

In [39]:
conflict_by_country.drop(columns="Unnamed: 0", inplace=True)
conflict_by_port.drop(columns="Unnamed: 0", inplace=True)

In [45]:
al_ahmadi = (
    conflict_by_port[conflict_by_port["portname"] == "Al Ahmadi"]
    .groupby(["portname", "date"])["nrEvents"]
    .sum()
    .reset_index()
)

In [57]:
data = (
    conflict_by_country[conflict_by_country["country"] == "Yemen"]
    .merge(bab_el_mandeb, on="date")
    .merge(al_ahmadi, on="date")
)
data.rename(
    columns={"nrEvents_x": "countryEvents", "nrEvents_y": "portEvents"}, inplace=True
)

In [69]:
from datetime import datetime

conflict_date = datetime(2023, 10, 7)
crisis_date = datetime(2023, 11, 17)

In [75]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Legend, Span

# Call this to display plots inline in a Jupyter Notebook
output_notebook()

data["date"] = pd.to_datetime(data["date"])


# Create a figure
p = figure(
    x_axis_type="datetime",
    plot_width=1000,
    plot_height=400,
    title="Comparing Weekly Trends in Conflict and Shipping",
)
p.title.text_font_size = "16pt"

# Plot multiple lines
p1 = p.line(data["date"], data["n_total"], line_width=2, color="#B07AA1")
p2 = p.line(data["date"], data["countryEvents"], line_width=2, color="#FF9DA7")
p3 = p.line(data["date"], data["portEvents"], line_width=2, color="#9C755F")

legend_items = [
    ("Number of Vessels", [p1]),
    ("Number of events in Yemen", [p2]),
    ("Number of events around Port Al Ahmadi", [p3]),
]

# Create the legend manually
legend = Legend(items=legend_items, location="center")

# Add the legend to the plot, but ensure it's outside the main figure
p.add_layout(legend, "right")

# Customize the plot
p.yaxis.axis_label = "Number of Vessels and Number of Conflict Events"
p.legend.location = "top_right"
p.yaxis.axis_label_text_font_style = "normal"

p.add_layout(
    Span(
        location=crisis_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)
p.add_layout(
    Span(
        location=conflict_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)

# Show the plot
show(p)

Loading BokehJS ...

### Exploring Correlation between Daily Trends of Conflict and Shipping

In [102]:
acled = pd.read_csv("../../data/conflict/acled_raw_2023-01-01_2024-02-27.csv")
acled["event_date"] = pd.to_datetime(acled["event_date"])
acled = acled.groupby(["country", "event_date"])["fatalities"].count().reset_index()

In [103]:
acled.rename(columns={"fatalities": "countryEvents", "event_date": "ymd"}, inplace=True)

In [99]:
df["ymd"] = pd.to_datetime(df["ymd"])

In [106]:
data = df[df["portid"] == "chokepoint4"].merge(
    acled[acled["country"] == "Yemen"], on="ymd"
)

In [117]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
import numpy as np

# Enable notebook output
output_notebook()


# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(data)

slope, intercept = np.polyfit(data["countryEvents"], data["n_total"], 1)
x = np.array(data["countryEvents"])
y_line = slope * x + intercept

# Create a ColumnDataSource for the line of best fit
line_source = ColumnDataSource(data=dict(x=x, y_line=y_line))

# Initialize Bokeh figure
p = figure(
    plot_width=800,
    plot_height=400,
    title="Correlation between Daily Conflict Events and Daily Vessels",
)
p.title.text_font_size = "16pt"

# Add scatter plot glyphs
p.circle(
    x="countryEvents", y="n_total", source=source, size=10, color="navy", alpha=0.5
)

p.line("x", "y_line", source=line_source, color="grey", line_width=2, line_dash=(4, 4))

# Customize plot
p.yaxis.axis_label = "Total Vessels"
p.xaxis.axis_label = "Conflict Events in the Country"

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

# Show plot
show(p)

Loading BokehJS ...